In [1]:
from keras import Sequential
import pandas
import sklearn
from sklearn.model_selection import train_test_split

from transformers import BertTokenizerFast, TFBertForSequenceClassification, TrainingArguments, Trainer

2022-03-16 23:59:48.937073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-16 23:59:48.937152: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Define Our Constants

In [ ]:
DATASETS_PATH = "../data"
DATASET_FILE_NAME = "hate-speech-labeled.csv"
TEST_SIZE = 0.2
TRAIN_SIZE = 0.8
DATA_NEED_PREPROCESSING = False
SENTENCE_MAX_LENGTH = 512
TOKENIZER_MODEL_NAME = "bert-base-uncased"
NUM_OF_LABELS = 3

### Reading Dataset

In [ ]:
loadedData = pandas.read_csv(DATASETS_PATH + "/" + DATASET_FILE_NAME)

loadedData.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


### Dataset Analysis

In [ ]:
print("Total Size: ", len(loadedData))
print("Test Size: ", len(loadedData) * TEST_SIZE)
print("Train Size: ", len(loadedData) * TRAIN_SIZE)
print("Does the data set have any null values? ", loadedData.isnull().values.any())
print("Does the data set have any duplicate values? ", loadedData.duplicated().any())
print("Does the data have any missing values? ", loadedData.isna().values.any())
print("Does the data have offensive words? ", loadedData["offensive_language"].any())
print("Number of hate speech tweets: ", (loadedData["class"] == 0).sum())
print("Number of neutral tweets: ", (loadedData["class"] == 2).sum())
print("Number of offensive tweets: ", (loadedData["class"] == 1).sum())

Total Size:  24783
Test Size:  4956.6
Train Size:  19826.4
Does the data set have any null values?  False
Does the data set have any duplicate values?  False
Does the data have any missing values?  False
Does the data have offensive words?  True
Number of hate speech tweets:  1430
Number of neutral tweets:  4163
Number of offensive tweets:  19190


### Preprocessing Data
(Some data are already pre-processed)

In [ ]:
##Preprocessing...
data = loadedData

tweetsText = data.tweet
tweetLabels = data['class']

### Splitting Data

In [ ]:
(trainTexts, testTexts, trainLabels, testLabels) = train_test_split(tweetsText, tweetLabels, test_size=TEST_SIZE, train_size=TRAIN_SIZE)

trainTexts = [str(text) for text in trainTexts]
testTexts = [str(text) for text in testTexts]
trainTexts

['@SexWith_SocksOn faggot',
 "RT @_twelvestarr: Niggas be catching attitudes and expect a bitch to beg plenty of sorry's to figure out what's his problem. NO",
 'Taking out a second mortgage and betting it all on the Yankees winning the World Cup. They win a lot; seemed like a no-brainer to me.',
 'RT @relatabIe: i hate when people ask &#8220;who you tryna look good for?!&#8221; bitch myself bye',
 "Imma treat you how you act ... If you act like a bitch you'll get treated like a bitch",
 '@Meeeesh1395 my fuckin mom is a bitch. Like she aint even payin for it. Dylan is and he said its fine.',
 "Lmao! You niggas regionally picky about hoes now. Just say anything on Twitter. Makes it's fun tho",
 '@IamSaintBryan they call me yellow bone',
 '@JaredReve use monkey maker you prick!',
 'RT @BriannDominguez: Gasoline - daddy Yankee &#128175;&#128076;',
 "RT @TheRoot: Don Lemon gets called an Uncle Tom, doesn't like it: http://t.co/eV4xgI5Pgc http://t.co/i6Xs8HCnrb",
 '#faggots are MERELY a #sc

## Using BERT

### Extracting BERT Encodings

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(TOKENIZER_MODEL_NAME, do_lower_case=True)

train_encodings = tokenizer(trainTexts, truncation=True, padding=True, max_length=SENTENCE_MAX_LENGTH)

### Extracting BERT Model

In [ ]:
model = TFBertForSequenceClassification.from_pretrained(TOKENIZER_MODEL_NAME, num_labels=NUM_OF_LABELS)

Downloading: 100%|██████████| 511M/511M [04:59<00:00, 1.79MB/s] 
2022-03-16 20:54:12.867164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-16 20:54:12.867269: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-16 20:54:12.867338: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ispenws-ubuntu): /proc/driver/nvidia/version does not exist
2022-03-16 20:54:12.915581: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-16 20:54:13.596025: W tensorflow/co

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    # logging_steps=200,
    evaluation_strategy="steps",
    output_dir="./saved"
)

ValueError: --load_best_model_at_end requires the saving steps to be a round multiple of the evaluation steps, but found 500, which is not a round multiple of 200.